# Data augmentation

## 1 Current data statistics

### We read in the files of queries, logical forms, and schema, and categorize them by length; within the same length, there would be subcategories

In [1]:
import numpy as np
import random
import math

logic_category_len = dict()
query_len = dict()
schema_len = dict()
with open('./rand.lo') as f_lo:
    with open('./rand.qu') as f_qu:
        with open('./rand.fi') as f_fi:
            logic_line, query_line, schema_line = f_lo.readline(), f_qu.readline(), f_fi.readline()
            while logic_line and query_line and schema_line:
                logic = logic_line.split()
#                 if len(logic) == 13:
#                     if logic[4] == 'less':
#                         logic[0] = 'argmax'
#                     else:
#                         logic[0] = 'argmin'
#                     logic.insert(2, logic[3])
                length = len(logic)
#                 if length ==0:
#                     continue
                if length not in logic_category_len:
                    logic_category_len[length] = []
                    query_len[length] = []
                    schema_len[length] = []
                logic_category_len[length].append(logic_line)
                query_len[length].append(query_line)
                schema_len[length].append(schema_line)
                logic_line, query_line, schema_line = f_lo.readline(), f_qu.readline(), f_fi.readline()
for key in logic_category_len.keys():
    value = logic_category_len[key]
    print 'length = %d, total examples: %d' %(key, len(value))

length = 2, total examples: 2
length = 4, total examples: 156
length = 6, total examples: 1253
length = 7, total examples: 4
length = 8, total examples: 624
length = 10, total examples: 687
length = 11, total examples: 697
length = 12, total examples: 488


Have a look at the data:

In [2]:
for i in range(len(logic_category_len[7])):
    print query_len[7][i]
    #print logic_category_len[14][i]

what is the difference between the nations with the most and least amount of bronze medals

how long in years has the this world series been occurring

what is the difference between the nations with the most and least amount of gold medals

what is the difference between the nations with the most and least amount of silver medals



### Now we collect all different schema in a list for later use

In [2]:
schema_collect = []
with open('./rand.fi') as f_fi:
    for line in f_fi:
        if line in schema_collect:
            continue
        schema_collect.append(line)
    
for schema in schema_collect:
    print schema

Nation Rank Gold Silver Bronze Total

Name Year_inducted Position Apps Goals

Year 1st_Venue 2nd_Venue 3rd_Venue 4th_Venue 5th_Venue 6th_Venue

Player Matches Innings Runs Average 100s 50s Games_Played Field_Goals Free_Throws Points

Team County Wins Years_won Areas Prices

Country Masters U.S._Open The_Open PGA Total

Swara Position Short_name Notation Mnemonic

State No._of_candidates No._of_elected Total_no._of_seats_in_Assembly Year_of_Election

Discipline Amanda Bernie Javine_H Julia Michelle

Nation Name Position League_Apps League_Goals FA_Cup_Apps FA_Cup_Goals Total_Apps Total_Goals

Menteri_Besar Took_office Left_office Party



## 2 Data Preparation and Generation

### Next we do some data generation, the first goal is to double our current data size (8k~10k) 

As we previously did some work in the file ./data_prep/categorization.txt, we have several different sentences for a single length category. For each sentence structure, we first see whether it could applied to all or several schema, or just a single schema; then we tag each sentence, and for 'field' and 'value', we do data recombination for both query and logical forms; finally we add noise and replace synonyms in the queries to further complicate the sentence structrue.

Let's start with the easiest length = 4:

In [3]:
import os,sys,inspect

import tagger as tg
import tag_utils as tu
from nltk.parse import stanford
from nltk import tree

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)
os.environ['STANFORD_PARSER'] = '/Users/richard_xiong/Documents/DeepLearningMaster/deep_parser'
os.environ['STANFORD_MODELS'] = '/Users/richard_xiong/Documents/DeepLearningMaster/deep_parser'

parser = stanford.StanfordParser(model_path='/Users/richard_xiong/Documents/DeepLearningMaster/deep_parser/englishPCFG.ser.gz')

#parsequery = "which nation has less than 6 <field:1> but its <field:2> medals are more than 14 "
#parsequery = "when the <field:1> was beijing and <field:2> was dubai , which city was the most recent <field:4>"
#parsequery = "for <field:0> with more than 400 <field:1> and <field:2> less than 14 , <field:0> has the most <field:3>"
# parsequery = "which state had the largest <field:1>, and its <field:2> are within 12 and 15"
# dependency_tree = parser.raw_parse_sents(('Hello, My name is Melroy', parsequery))

# for line in dependency_tree[1]:
#     line.draw()

Importing GloVe pretrained word vectors
		reading 10000 lines from GloVe file
		reading 20000 lines from GloVe file
		reading 30000 lines from GloVe file
		reading 40000 lines from GloVe file
		reading 50000 lines from GloVe file
Replacing GloVe word vectors as initialization


In order to find which value corresponds to which field, we need to first find:
1. the lowest common ancestor for each (value, field) pairs
2. for each value, all different ancestors are belong to different levels, the deepest one, which should be the subtree for all the others, would contain the correspondence pair

Possible functions:

leaf_treeposition(self, index) ---> return: The tree position of the ``index``-th leaf in this
            tree.  I.e., if ``tp=self.leaf_treeposition(i)``, then
            ``self[tp]==self.leaves()[i]``.

treeposition_spanning_leaves(self, start, end) ---> The tree position of the lowest descendant of this
            tree that dominates ``self.leaves()[start:end]``.

convert(cls, tree) ---> to subtype of Tree, say, ParentTree

e.g.
(0, 0, 1, 0, 0, 1, 0)
(0, 0, 1, 0, 1, 1, 0, 0)
(0, 0, 1, 2, 0, 0, 0)
(0, 0, 1, 2, 1, 1, 0, 0)

In [4]:
schema_collect[2] = "State Year_of_Election No._of_candidates No._of_elected Total_no._of_seats_in_Assembly \n"
schema_collect[7] = "Year 1st_Venue 2nd_Venue 3rd_Venue 4th_Venue 5th_Venue 6th_Venue \n"
schema_collect[3] = "Team Years_won County Wins Areas Prices \n"
schema_collect[4] = "Player Matches Innings Runs Average 100s 50s Games_Played Field_Goals Free_Throws Points \n"

schema_collect[6] = "Discipline Amanda Bernie Javine_H Julia Michelle \n"
schema_collect[8] = "Swara Position Short_name Notation Mnemonic \n"
schema_collect[7] = "Nation Name Position League_Apps League_Goals FA_Cup_Apps FA_Cup_Goals Total_Apps Total_Goals \n"
schema_collect[9] = "Year 1st_Venue 2nd_Venue 3rd_Venue 4th_Venue 5th_Venue 6th_Venue \n"

for schema in schema_collect:
    print schema

Nation Rank Gold Silver Bronze Total

Name Year_inducted Position Apps Goals

State Year_of_Election No._of_candidates No._of_elected Total_no._of_seats_in_Assembly 

Team Years_won County Wins Areas Prices 

Player Matches Innings Runs Average 100s 50s Games_Played Field_Goals Free_Throws Points 

Country Masters U.S._Open The_Open PGA Total

Discipline Amanda Bernie Javine_H Julia Michelle 

Nation Name Position League_Apps League_Goals FA_Cup_Apps FA_Cup_Goals Total_Apps Total_Goals 

Swara Position Short_name Notation Mnemonic 

Year 1st_Venue 2nd_Venue 3rd_Venue 4th_Venue 5th_Venue 6th_Venue 

Menteri_Besar Took_office Left_office Party



### Conventions 
1. "o" stands for "ordinal" values, refering to schema_collect[0:4]
2. "n" stands for "numerical" values, refering to schema_collect[4:8]
3. "s" stands for "string" values, refering to schema_collect[7:]

In [5]:
collect4max = """which country has the most pga
which country has the most pga championships
which country had the most number of wins
which country won the largest haul of bronze medals
who was the last de player
which nation received the largest amount of gold medals
the team with the most gold medals
which nation was ranked last
the country that won the most medals was
what is the largest matches amount""".split('\n')

collect4min = """who was the first nation
what is the name of the first nation on this chart
what is the name of the swara that holds the first position
which country had the least bronze medals
who scored the least on whitewater_kayak
which state has the top no._of_elected amount
who was the top scorer in innings
what is the top listed player
who is the top ranked nation""".split('\n')

collect6select = """what are the number of league_apps ted_davis has
what is the name of the only nation that did not earn any bronze medals
what are the number of pga winning golfers that zimbabwe has
who only won 13 silver medals
which country was awarded more than 5 silver medals
what is the number of wins for maynooth
what was the number of silver medals the ivory_coast won
only team to have more than 30 medals
how many u.s._open wins does fiji have
which country won only one medal, a bronze medal
which ranking is mexico
who won more gold medals than the united_states
name a player whose average was above 25
how many silver medals did brazil received
name a player that play in no more than 13 innings
what country has won no silver medals
before 1990, how many pga were obtained
what's the number of silver medals did chile win""".split('\n')

# How to deal with no/not which indicates zero?
# The currect augmentation directly copy field name, could extend to query words
# even the field not shown in the query, the current algorithm still works

# print collect6select
# print collect4max

lo4max = 'select <field>:0 argmax <field>:1'
lo4min = 'select <field>:0 argmin <field>:1'
lo6select = 'select <field>:0 where <field>:1 equal <value>:1'

print schema_collect[5], collect6select[2]
tagged2, field_corr, value_corr, quTemp, _ = \
            tg.sentTagging_tree(parser, collect6select[2], schema_collect[5])
print field_corr
print value_corr 
print quTemp
print lo6select

Country Masters U.S._Open The_Open PGA Total
what are the number of pga winning golfers that zimbabwe has
['<field>', 'PGA']
['<value>', 'Country']
[(5, 0)]
[]
[]
[]
PGA Country
<nan> zimbabwe
what are the number of <field>:0 winning golfers that <value>:1 has
select <field>:0 where <field>:1 equal <value>:1


### Note:
Each sentence could then be turned into a query tempelate after tagging. Now we have the logical template, query template, and several available schema (annotated by 'o' 'n' 's'), so combined with the field_corr and value_corr files we should be able to generate multiple sentences according to several schema.

In [9]:
field_corr_dicts = [{'County': ['Louth', 'Dublin', 'Kildare', 'Laois', 'Wicklow'], 'Team': ['Ireland', 'Spain', 'Cyprus', 'Mexico', 'Maynooth']}, \
                    {'Prices': [28, 62, 72, 9, 40], 'Wins': [52, 80, 42, 76, 29], 'Areas': [38, 1, 7, 83, 98]}]

def isRepetitive(sequence):
    for element in sequence[:-1]:
        if element == sequence[-1]:
            return True
    return False

def generateFieldCombs(field_corr_dicts):
    ''' If only fields are recombinable'''
    list_of_seqs = []
    if len(field_corr_dicts) == 1:
        # base case:
        for key in field_corr_dicts[0].keys():
            list_of_seqs.append([key])
    else:
        # recursive case:
        former_seqs = generateFieldCombs(field_corr_dicts[:-1])
        for key in field_corr_dicts[-1].keys():
            for seq in former_seqs:
                newseq = [x for x in seq]
                newseq.append(key)
                # check new repetitive elements
                if not isRepetitive(newseq):
                    list_of_seqs.append(newseq)
    return list_of_seqs

def generateValueCombs(field_corr_dicts, field_combination, qu_value):
    ''' Both fields and values are recombinable
        arguments --- field_combination: the selected field combination, where the value are to be decided
    '''
    list_of_seqs = []
    if len(qu_value) == 1:
        # base case:
        _, idx = qu_value[0]  # check position of values
        for value in field_corr_dicts[idx][field_combination[idx]]:
            list_of_seqs.append([value])
    else:
        # recursive case:
        former_seqs = generateValueCombs(field_corr_dicts, field_combination, qu_value[:-1])
        _, idx = qu_value[-1]
        for value in field_corr_dicts[idx][field_combination[idx]]:
            for seq in former_seqs:
                newseq = [x for x in seq]
                newseq.append(value)
                # check new repetitive elements
                if not isRepetitive(newseq):
                    list_of_seqs.append(newseq)
    return list_of_seqs

print generateFieldCombs(field_corr_dicts)
print generateValueCombs(field_corr_dicts, ['County', 'Wins'], [(2,0), (4,1)])

[['County', 'Prices'], ['Team', 'Prices'], ['County', 'Wins'], ['Team', 'Wins'], ['County', 'Areas'], ['Team', 'Areas']]
[['Louth', 52], ['Dublin', 52], ['Kildare', 52], ['Laois', 52], ['Wicklow', 52], ['Louth', 80], ['Dublin', 80], ['Kildare', 80], ['Laois', 80], ['Wicklow', 80], ['Louth', 42], ['Dublin', 42], ['Kildare', 42], ['Laois', 42], ['Wicklow', 42], ['Louth', 76], ['Dublin', 76], ['Kildare', 76], ['Laois', 76], ['Wicklow', 76], ['Louth', 29], ['Dublin', 29], ['Kildare', 29], ['Laois', 29], ['Wicklow', 29]]


In [ ]:
def schemaRecommend(field_corr_old, special_code):
    ''' From the old generated field correspondence (string), transform to a new field correspondence, 
        represented by a list value_types, and from the set of value types to get the possible schemas 
        (PLURALS) that could use for augmentation later (check that all the types in field_corr_new 
        should be in each schema)
        arguments --- special_code: might be used to indicate that the schema is not tranferrable. 
                      default False, means able to generalize to schema that contain corresponding field 
                      types; if True, means only applicable to original schema.
        return --- field_corr: a list of value_types
                   schemas: several schema that the template could augment to, each contain all the
                   value_types needed; also see 'special_code'.
    '''
    return field_corr_new, schemas



In [11]:
field_corr_new = ['int','string']
schema_aug = schema_collect[0:8]

def augment4max(quTemp, loTemp, field_corr, schema_aug):
    ''' Data augmentation from a pair of query template and logical template
        arguments --- field_corr: a list of value_types e.g. ['string','ordinal','int'], each idx should 
                      correspond to the postion in the templates
                      schemas: PLURALS HERE! several schemas that the template could augment to.
        return --- collections of queries, logics, and fields
    '''
    queryCollect, logicCollect, fieldCollect = [], [], []
    config = tu.Config()
    
    # Step 1: preparation
    query = quTemp.split()
    logic = loTemp.split()
    qu_field = []  # positions of field in query
    qu_value = []  # positions of value in query
    lo_field = []  # positions of field in logic
    lo_value = []  # positions of value in logic
    for i in range(len(query)):
        reference = query[i].split(':')
        if len(reference) == 1:
            continue
        print reference
        idx = int(reference[1])
        if reference[0] == '<field>':
            qu_field.append((i, idx))
        else:
            qu_value.append((i, idx))
    print qu_field, qu_value
    for i in range(len(logic)):
        reference = logic[i].split(':')
        if len(reference) == 1:
            continue
        print reference
        idx = int(reference[1])
        if reference[0] == '<field>':
            lo_field.append((i, idx))
        else:
            lo_value.append((i, idx))
    print lo_field, lo_value
    
    # Step 2: augment to different schemas
    for j in range(len(schema_aug)):
        # Step 2.1: for each schema, build correspondence list of dictionarys: [{}, {}, {}]
        field_corr_dicts = []
        print '=== %d schema ===' %j
        schema = schema_aug[j].split()
        #print schema
        # because there could be multiple same-type fields in one sentences, we go over field_corr
        for k in range(len(field_corr)):
            field_corr_dict = dict()
            for i in range(len(schema)):
                field = schema[i]
                #print field
                if schema[i] == 'Total' or schema[i] == 'Average':
                    continue
                value_type = config.field2word[schema[i]]['value_type']
                #print value_type
                if value_type == field_corr[k]:
                    if value_type == 'string':
                        #field_corr_dict[field] = config.field2word[schema[i]]['value_range']
                        field_corr_dict[field] = random.sample(config.field2word[schema[i]]['value_range'], 5)
                    elif value_type == 'int':
                        field_corr_dict[field] = [random.randint(1, 99) for i in range(5)]
                    elif value_type == 'date':
                        field_corr_dict[field] = [random.randint(1970, 2010) for i in range(5)]
                    elif value_type == 'ordinal':
                        field_corr_dict[field] = [random.randint(1, 9) for i in range(3)]
            field_corr_dicts.append(field_corr_dict)
        print field_corr_dicts 
        # now the list of dicts [{str_field1:[], str_field2:[], ...}, {int_field1:[], int_field2:[], ...}]
        
        # Step 2.2: Regenerate sentence by filling into the place
        field_combinations = generateFieldCombs(field_corr_dicts)
        for field_combination in field_combinations:
            newquery = [x for x in query]
            newlogic = [x for x in logic]
            # regenerate query, lower case or query_word
            for (posit, idx) in qu_field:
                field_info = config.field2word[field_combination[idx]]
                if len(field_info['query_word']) > 1:
                    pick = random.choice(field_info['query_word'])
                    while pick == 'who' or pick == 'when' or pick == 'city':
                        pick = random.choice(field_info['query_word'])
                    newquery[posit] = pick
                else:
                    newquery[posit] = field_combination[idx].lower()                
            # regenerate logic forms
            for (posit, idx) in lo_field:
                newlogic[posit] = field_combination[idx]
            if len(qu_value) > 0:
                value_combinations = generateValueCombs(field_corr_dicts, field_combination, qu_value)
                for value_combination in value_combinations:
                    print '*** Value Augmentation ***'
                    print value_combination
                    print qu_value
                    print lo_value
                    morequery = [x for x in newquery]
                    morelogic = [x for x in newlogic]
                    for (posit, idx) in qu_value:
                        morequery[posit] = value_combination[idx].lower()
                    for (posit, idx) in lo_value:
                        morelogic[posit] = value_combination[idx]
                    queryCollect.append(' '.join(morequery))
                    logicCollect.append(' '.join(morelogic))
                    fieldCollect.append(schema_aug[j])
                continue
            queryCollect.append(' '.join(newquery))
            logicCollect.append(' '.join(newlogic))
            fieldCollect.append(schema_aug[j])
    return queryCollect, logicCollect, fieldCollect, 

augment4max(quTemp, lo6select, field_corr_new, schema_aug)

['<field>', '0']
['<value>', '1']
[(5, 0)] [(9, 1)]
['<field>', '0']
['<field>', '1']
['<value>', '1']
[(1, 0), (3, 1)] [(5, 1)]
=== 0 schema ===
[{'Bronze': [93, 89, 35, 48, 44], 'Silver': [5, 55, 67, 51, 85], 'Gold': [75, 23, 94, 85, 49]}, {'Nation': ['Latvia', 'Hungary', 'Austria', 'US', 'Netherlands']}]
*** Value Augmentation ***
['Latvia']
[(9, 1)]
[(5, 1)]


IndexError: list index out of range

In [18]:
key = 'nation'
turn = [key]
print turn

['nation']
